In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

/var/folders/4c/sl3jt_wj26dgbkh3sqt9fmcm0000gn/T/ipykernel_14312/3777615979.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


# Lab | Natural Language Processing
### SMS: SPAM or HAM

### Let's prepare the environment

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer

- Read Data for the Fraudulent Email Kaggle Challenge
- Reduce the training set to speead up development. 

In [93]:
## Read Data for the Fraudulent Email Kaggle Challenge
data = pd.read_csv("/Users/manish/Documents/Ironhack/Week 7/W7D1/Lab- NLP/lab-natural-language-processing/data/kg_train.csv",encoding='latin-1')

# Reduce the training set to speed up development. 
# Modify for final system
data = data.head(1000)
print(data.shape)
data.fillna("",inplace=True)

(1000, 2)


In [94]:
data.head()

,text,label
0,"DEAR SIR, STRICTLY A PRIVATE BUSINESS PROPOSAL...",1
1,Will do.,0
2,Nora--Cheryl has emailed dozens of memos about...,0
3,Dear Sir=2FMadam=2C I know that this proposal ...,1
4,fyi,0


### Let's divide the training and test set into two partitions

In [95]:
# Your code
from sklearn.model_selection import train_test_split
data_train, data_valid = train_test_split(data, test_size=0.2, random_state= 42, stratify= data['label'])


## Data Preprocessing

In [20]:
%pip install nltk

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 25.0.1 -> 26.0.1
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [96]:
import string
from nltk.corpus import stopwords
print(string.punctuation)
print(stopwords.words("english")[100:198])
from nltk.stem.snowball import SnowballStemmer
snowball = SnowballStemmer('english')
len(stopwords.words('English'))

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
['needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 're', 's', 'same', 'shan', "shan't", 'she', "she'd", "she'll", "she's", 'should', 'shouldn', "shouldn't", "should've", 'so', 'some', 'such', 't', 'than', 'that', "that'll", 'the', 'their', 'theirs', 'them', 'themselves', 'then', 'there', 'these', 'they', "they'd", "they'll", "they're", "they've", 'this', 'those', 'through', 'to', 'too', 'under', 'until', 'up', 've', 'very', 'was', 'wasn', "wasn't", 'we', "we'd", "we'll", "we're", 'were', 'weren', "weren't", "we've", 'what', 'when', 'where', 'which', 'while', 'who', 'whom', 'why', 'will', 'with', 'won', "won't", 'wouldn', "wouldn't", 'y', 'you', "you'd", "you'll", 'your', "you're", 'yours', 'yourself', 'yourselves', "you've"]


198

In [97]:
target_train = data_train['label']
data_train = data_train.drop('label', axis = 1)
target_valid = data_valid['label']
data_valid = data_valid.drop('label', axis = 1)

In [98]:
data_train['text'][:-5]

442    Dear=2C Good day hope fine=2Cdear am writting ...
962    FROM MR HENRY KABORETHE CHIEF AUDITOR INCHARGE...
971                                             Will do.
190    FROM THE DESK OF DR.ADAMU  ISMALERAUDITING AND...
551    Dear Friend, My name is LOI C.ESTRADA,The wife...
                             ...                        
569    DQpEZWFyIFNpci9NYWRhbSwNCiANCkkgYW0gTXIuIElicm...
841    When will he do Senate/Feinstein?From Mills Ch...
732    Dear Sir/Madam,I am Mr. Lee Yung, managing Hub...
513    I sent call sheet for indonesia for marina to ...
220    FYI Ã¢ÂÂ today the SFRC will take up at its ...
Name: text, Length: 795, dtype: object

## Now, we have to clean the html code removing words

- First we remove inline JavaScript/CSS
- Then we remove html comments. This has to be done before removing regular tags since comments can contain '>' characters
- Next we can remove the remaining tags

In [99]:
len(data_train)

800

In [125]:
# Your code
# 1) Remove inline JavaScript/CSS blocks
SCRIPT_STYLE_RE = re.compile(
    r"(?is)<(script|style)\b[^>]*>.*?</\1\s*>"
)
# (?i) case-insensitive, (?s) dot matches newlines
# 2) Remove HTML comments (do this before tags)
COMMENTS_RE = re.compile(
    r"(?s)<!--.*?-->"
)
# non-greedy so it stops at the first -->
# 3) Remove remaining tags
TAGS_RE = re.compile(
    r"(?s)<[^>]+>"
)
# Optional: clean up whitespace after stripping
WHITESPACE_RE = re.compile(r"\s+")
def strip_html_regex(html: str) -> str:
    # Remove script/style blocks
    text = SCRIPT_STYLE_RE.sub("", html)
    # Remove comments
    text = COMMENTS_RE.sub("", text)
    # Remove remaining tags
    text = TAGS_RE.sub("", text)
    # Normalize whitespace (optional, but usually helpful)
    text = WHITESPACE_RE.sub(" ", text).strip()
    return text

In [126]:
for i in range(len(data_train)):
    data_train.iloc[i] = strip_html_regex(str(data_train.iloc[i]))

In [127]:
for i in range(len(data_valid)):
    data_valid.iloc[i] = strip_html_regex(str(data_valid.iloc[i]))

In [128]:
data_train.iloc[0]

text    text text text text 442 dear=2c good day hope....
Name: 442, dtype: object

In [101]:
from string import punctuation
print(set(punctuation))

{'+', "'", ':', '^', '}', '/', '"', '_', '%', '@', '\\', '&', '<', '$', '>', ';', '(', '.', ',', '#', '{', '[', '*', ')', '|', '`', '?', ']', '=', '~', '-', '!'}


In [ ]:
#data_train['text'] = data_train['text'].str.replace(r'https:/w{3}/.')

In [102]:
data_train['text'] = data_train['text'].str.replace(r'\d[0-9]', ' ', regex = True)

In [112]:
data_train['text'] = data_train['text'].str.replace('=2c', ' ')

In [115]:
data_train['text'] = strip_html_regex(str(data_train))

In [119]:
data.iloc[442]

text     Dear=2C Good day hope fine=2Cdear am writting ...
label                                                    1
Name: 442, dtype: object

In [116]:
data_train['text']

442    text 442 Dear=2C Good day hope fine=2Cdear am ...
962    text 442 Dear=2C Good day hope fine=2Cdear am ...
971    text 442 Dear=2C Good day hope fine=2Cdear am ...
190    text 442 Dear=2C Good day hope fine=2Cdear am ...
551    text 442 Dear=2C Good day hope fine=2Cdear am ...
                             ...                        
625    text 442 Dear=2C Good day hope fine=2Cdear am ...
53     text 442 Dear=2C Good day hope fine=2Cdear am ...
728    text 442 Dear=2C Good day hope fine=2Cdear am ...
664    text 442 Dear=2C Good day hope fine=2Cdear am ...
798    text 442 Dear=2C Good day hope fine=2Cdear am ...
Name: text, Length: 800, dtype: object

In [113]:
data_train

,text
442,Dear=2C Good day hope fine=2Cdear am writting ...
962,FROM MR HENRY KABORETHE CHIEF AUDITOR INCHARGE...
971,Will do.
190,FROM THE DESK OF DR.ADAMU ISMALERAUDITING AND...
551,"Dear Friend, My name is LOI C.ESTRADA,The wife..."
...,...
625,"<P><A href=3D""http://www.rte.ieews/ / /zimba..."
53,"IV><DIV>Dear,<BR>&nbsp;</DIV><DIV>It is my ple..."
728,Will do. Think we are in good shape. Immelt In...
664,With your permission we are changing the state...


In [ ]:
pattern = re.complie('https?://(?:www\.?)\w+\.\w{2,}')

In [129]:
punct_pattern = f"[{re.escape(string.punctuation)}]"
#data_train['text'] = data_train['text'].str.replace(punct_pattern, '', regex = True) # to remove all the special characters
import re
for i in range(len(data_valid)):
    data_valid.iloc[i] = data_valid.iloc[i].str.replace(punct_pattern, ' ', regex = True)   
    data_valid.iloc[i] = re.sub(r'\d[0-9]', ' ', str(data_valid.iloc[i])) # to remove any digit
    data_valid.iloc[i] = str(data_valid.iloc[i]).replace('text', ' ') # to remove any single character
    data_valid.iloc[i] = data_valid.iloc[i].str.lower()
    #data_train.iloc[i] = re.sub(r'\w['text']', '', str(data_train.iloc[i]))                         
#    for item in set(punctuation):                            
#        data_train.iloc[i] = re.sub(r'\[item]', '', str(data_train.iloc[i])) # to remove all special Characters 
#for item in data_train['text']:
#    item = re.sub(r'\d[0-9]', '', item)

data_valid

,text
222,strictly personalpermit me to int....
824,dear nancy i very much want to m....
505,request for business partnershipi....
165,soneri bank limitedsilver jubilee....
832,dear partner i am a fund manager ....
...,...
459,fyi name 9 dtype object\nname....
541,dear sir 2c i got your contact in....
905,hello i am very sorry to embarras....
219,we are meeting with you at noon t....


In [89]:
# Your code
punct_pattern = f"[{re.escape(string.punctuation)}]"
#data_train['text'] = data_train['text'].str.replace(punct_pattern, '', regex = True) # to remove all the special characters
import re
for i in range(len(data_train)):
    data_train.iloc[i] = data_train.iloc[i].str.replace(punct_pattern, ' ', regex = True)   
    data_train.iloc[i] = re.sub(r'\d[0-9]', ' ', str(data_train.iloc[i])) # to remove any digit
    data_train.iloc[i] = str(data_train.iloc[i]).replace('text', ' ') # to remove any single character
    data_train.iloc[i] = data_train.iloc[i].str.lower()
    #data_train.iloc[i] = re.sub(r'\w['text']', '', str(data_train.iloc[i]))                         
#    for item in set(punctuation):                            
#        data_train.iloc[i] = re.sub(r'\[item]', '', str(data_train.iloc[i])) # to remove all special Characters 
#for item in data_train['text']:
#    item = re.sub(r'\d[0-9]', '', item)

data_train

,text
442,dear2c good day hope fine2cd...
962,from mr henry kaborethe chie...
971,will do nname 1 dtype obj...
190,from the desk of dradamu is...
551,dear friend my name is loi c...
...,...
625,pa href3dhttpwwwrteieewszimb...
53,ivdivdearbrnbspdivdivit is m...
728,will do think we are in good...
664,with your permission we are ...


- Remove all the special characters
    
- Remove numbers
    
- Remove all single characters
 
- Remove single characters from the start

- Substitute multiple spaces with single space

- Remove prefixed 'b'

- Convert to Lowercase

In [ ]:
# Your code

## Now let's work on removing stopwords
Remove the stopwords.

## Tame Your Text with Lemmatization
Break sentences into words, then use lemmatization to reduce them to their base form (e.g., "running" becomes "run"). See how this creates cleaner data for analysis!

In [120]:
# Your code
from nltk.stem.porter import PorterStemmer
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer

porter   = PorterStemmer()
snowball = SnowballStemmer('english')
wordnet_lemma  = WordNetLemmatizer()

In [124]:
#porter.stem(data_train)
for i in range(len(data_train)):
    data_train.iloc[i] = snowball.stem(str(data_train.iloc[i]))
    data_train.iloc[i] = wordnet_lemma.lemmatize(str(data_train.iloc[i]), pos = 'v')
#snowball.stem(data_train)
#wordnet_lemma.lemmatize(data_train)
#wordnet_lemma.lemmatize(data_train,pos="v")

In [130]:
for i in range(len(data_valid)):
    data_valid.iloc[i] = snowball.stem(str(data_valid.iloc[i]))
    data_valid.iloc[i] = wordnet_lemma.lemmatize(str(data_valid.iloc[i]), pos = 'v')

## Bag Of Words
Let's get the 10 top words in ham and spam messages (**EXPLORATORY DATA ANALYSIS**)

In [ ]:
from collections import Counter 
for 

## Extra features

In [132]:
# We add to the original dataframe two additional indicators (money symbols and suspicious words).
money_simbol_list = "|".join(["euro","dollar","pound","€",r"\$"])
suspicious_words = "|".join(["free","cheap","sex","money","account","bank","fund","transfer","transaction","win","deposit","password"])

data_train['money_mark'] = data_train['text'].str.contains(money_simbol_list)*1
data_train['suspicious_words'] = data_train['text'].str.contains(suspicious_words)*1
data_train['text_len'] = data_train['text'].apply(lambda x: len(x)) 

data_valid['money_mark'] = data_valid['text'].str.contains(money_simbol_list)*1
data_valid['suspicious_words'] = data_valid['text'].str.contains(suspicious_words)*1
data_valid['text_len'] = data_valid['text'].apply(lambda x: len(x)) 

data_train.head()

,text,money_mark,suspicious_words,text_len
442,text text text text 442 dear=2c good day hope....,0,0,73
962,text text text text 442 dear=2c good day hope....,0,0,73
971,text text text text 442 dear=2c good day hope....,0,0,73
190,text text text text 442 dear=2c good day hope....,0,0,73
551,text text text text 442 dear=2c good day hope....,0,0,73


## How would work the Bag of Words with Count Vectorizer concept?

In [133]:
headlines = []
for row in range(0,len(data_train.index)):
    headlines.append(' '.join(str(x) for x in data_train.iloc[row,0:1]))
headlines1 = []
for row in range(0,len(data_valid.index)):
    headlines1.append(' '.join(str(x) for x in data_valid.iloc[row,0:1]))    

In [135]:
# Your code
from sklearn.feature_extraction.text import CountVectorizer
countvector=CountVectorizer(ngram_range=(2,2))
traindataset=countvector.fit_transform(headlines)
testdataset = countvector.transform(headlines1)

## TF-IDF

- Load the vectorizer

- Vectorize all dataset

- print the shape of the vetorized dataset

In [137]:
# Your code


tfidfvector=TfidfVectorizer(ngram_range=(2,2))
traindataset=tfidfvector.fit_transform(headlines)
testdataset = tfidfvector.transform(headlines1)

## And the Train a Classifier?

In [138]:
# Your code
from sklearn.ensemble import RandomForestClassifier
randomclassifier=RandomForestClassifier(n_estimators=200,criterion='entropy')
randomclassifier.fit(traindataset,target_train)

RandomForestClassifier(criterion='entropy', n_estimators=200)

In [140]:
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
predictions = randomclassifier.predict(testdataset)
matrix=confusion_matrix(target_valid,predictions)
print(matrix)
score=accuracy_score(target_valid,predictions)
print(score)
report=classification_report(target_valid,predictions)
print(report)

[[112   0]
 [ 88   0]]
0.56
              precision    recall  f1-score   support

           0       0.56      1.00      0.72       112
           1       0.00      0.00      0.00        88

    accuracy                           0.56       200
   macro avg       0.28      0.50      0.36       200
weighted avg       0.31      0.56      0.40       200



/Users/manish/Library/Python/3.9/lib/python/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/manish/Library/Python/3.9/lib/python/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/manish/Library/Python/3.9/lib/python/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(resu

### Extra Task - Implement a SPAM/HAM classifier

https://www.kaggle.com/t/b384e34013d54d238490103bc3c360ce

The classifier can not be changed!!! It must be the MultinimialNB with default parameters!

Your task is to **find the most relevant features**.

For example, you can test the following options and check which of them performs better:
- Using "Bag of Words" only
- Using "TF-IDF" only
- Bag of Words + extra flags (money_mark, suspicious_words, text_len)
- TF-IDF + extra flags


You can work with teams of two persons (recommended).

In [ ]:
# Your code